# CityBike Wien Datensetanalye für die Jahre 2016 und 2017

== Inhalt und Struktur dieses Notebooks 



## Datenset importieren bereinigen und transformieren

Die Daten werden zuerst eingelesen und nötige Transformationen werden vorgenommen. 

In erste Linie werden alle Fahrten die unter zwei Minuten dauern und an der gleichen Station enden aus dem Datenset herausgenommen da man davon ausgehen kann, dass der Nutzer das System nicht benutzt hat. 

In [46]:
%matplotlib inline
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df2016 = pd.read_csv('CityBikeWien/2016trips.csv', error_bad_lines=False, parse_dates=['Startdatum', 'Enddatum'], delimiter=';')
df2017 = pd.read_csv('CityBikeWien/2017trips.csv', error_bad_lines=False, parse_dates=['Startdatum', 'Enddatum'], delimiter=';')

Nachdem die beiden DataFrames konkateniert wurden muss der Index neu gesetzt werden

In [28]:
df_total = df2016.append(df2017)
df_total.reset_index(drop=True)

Neue Spalte für die Fahrtdauer hinzufügen

In [ ]:
df_total['Fahrtdauer'] =  df_total['Enddatum'] - df_total['Startdatum']

Uns interessieren nur Fahrten die länger als zwei Minuten dauern und gleichzeitig nicht an der gleichen Station beendet wurden

In [32]:
df_total_cleansed = df_total[ (df_total['Fahrtdauer'] > pd.Timedelta('120 sec')) 
                            & (df_total['Anfangsstation_ID'] != df_total['Endstation_ID'])]

Das bereinigte DataFrame enthält insgesamt 1.897.764 Einträge für die Jahre 2016 und 2017 - Siehe Beispiel

In [49]:
df_total_cleansed.info()
df_total_cleansed.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1897764 entries, 1 to 1013368
Data columns (total 7 columns):
Startdatum           datetime64[ns]
Enddatum             datetime64[ns]
Anfangsstation_ID    int64
Endstation_ID        int64
Fahrrad_ID           int64
Subscriber_ID        object
Fahrtdauer           timedelta64[ns]
dtypes: datetime64[ns](2), int64(3), object(1), timedelta64[ns](1)
memory usage: 155.8+ MB


,Startdatum,Enddatum,Anfangsstation_ID,Endstation_ID,Fahrrad_ID,Subscriber_ID,Fahrtdauer
1,2016-01-01 00:13:04,2016-01-01 00:38:05,1026,1100,2539,0x04084286D0CC7FA3973C16F8306802A1,00:25:01


## Finde die ersten 50 Benutzer nach Benutzung für den gesamten Zeitraum (2016+2017)


In der folgenden Zelle wird das DataFrame anhand der 'Subscriber_ID' gruppiert. 

Darauffolgend wird eine Aggregatfunktion (np.size) auf das gruppierte Objekt angewendet um die Anzahl der Fahrten je Subscriber zu erhalten. Somit erhält man ein Series-Objekt welches sortiert werden muss. 

Um die Handhabung zu erleichtern wird das Series-Objekt zu einem DataFrame mit einer einzigen Spalte transformiert.

In [51]:
grouped_by_subscriber_groups              = df_total.groupby('Subscriber_ID')
aggregated_by_subscriber_series           = grouped_by_subscriber_groups['Fahrrad_ID'].agg(np.size)
aggregated_sorted_by_subscriber_series    = aggregated_by_subscriber_series.sort_values(ascending=False)
aggregated_by_subscriber_dataframe        = aggregated_sorted_by_subscriber_series.to_frame('ANZAHL_FAHRTEN')

Die ersten fünfzig Subscriber werden extrahiert und zudem wird eine Spalte mit dem Rang hinzugefügt. Danach wird das DataFrame ausgegeben. 

In [52]:
aggregated_by_subscriber_dataframe_top_50 = aggregated_by_subscriber_dataframe[:50]
aggregated_by_subscriber_dataframe_top_50['RANG'] = np.arange(1,51) 
aggregated_by_subscriber_dataframe_top_50

C:\Users\jzvonek\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ANZAHL_FAHRTEN,RANG
Subscriber_ID,,
0x30EF7C4D0C1211B47ADBA3295EF47C61,1790,1
0xB9AF912E74C4F6602CF849721E28B621,1400,2
0xAC44D643FD2F8AE0CCD64B35A269DCC1,1297,3
0x30F9A85D6ADB8CFA380C4EA7D77C574E,1281,4
0xCA316F5E99321FD79D33C3CAB1000DA5,1193,5
0xE88FAEE35C9817B6BD91961D8DD186E0,1028,6
0x62B4268B5D5AB0FD8B0FDC93D79498DD,1025,7
0x24384492DBD8ACA63F4C29FC8BC3B36A,1011,8
0xD372854C5186AB811AEE55325547CC85,1009,9


## Was hat der Benutzer mit den meisten Fahrten für die Nutzung des Systems bezahlt?

Der 'Citybike Wien'-Nutzer welcher das System innerhalb der beiden Jahren am meisten genutzt hat, fuhr insgesamt 1790 Fahrten. Es wäre interessant zu sehen was ihn die Nutzung des Systems über die Zeitdauer der beiden Jahre gekostet hat. 

Hierfür ist es nötig den Kostenplan von Citybike Wien zu kennen. Dieser ist progressiv eingestellt, das heißt, dass die erste Stunde kostenlos ist. Die zweite Stunde kostet einen Euro, die dritte das doppelte und die vierte Stunde dann vier Euro. Jede weitere Stunde kostet jeweils vier Euro. 

Wir suchen alle Fahrten für den Power-User heraus

In [53]:
df_user_1 = df_total[df_total['Subscriber_ID'] == '0x30EF7C4D0C1211B47ADBA3295EF47C61']

Wie oft hat der Nutzer das Limit der einen Stunde überschritten? Insgesamt 98 mal.

In [60]:
df_user_1[df_user_1['Fahrtdauer'] > pd.Timedelta('60 min')].size

98

Gab es Fälle bei denen die Fahrt länger als zwei Stunden dauerte? Nein

In [63]:
df_user_1[df_user_1['Fahrtdauer'] > pd.Timedelta('120 min')].size

0

Insgesamt fielen 98 kostenpflichtige Fahrten an welche weniger als zwei Stunden gedauert haben. Somit hat der Anwender insgesamt 98 Euro für die Nutzung des Systems bezahlt.

Im Schnitt hat der oder die NutzerIn ~ 5,5 Cent für jede Fahrt bezahl.


## Welchen Umsatz hat das System für das Jahr 2016 bzw. 2017 generiert?

Es wäre interessant zu sehen wie viel Umsatz alleine durch die Nutzung des Systems entstanden ist. 

Der Betreiber von CityBike Wien, die Gewista GmbH wählt ein hybrides Finanzierungsmodell. Das heißt, dass sich das System nicht nur aus den Beiträgen der einzelnen Nutzer finanziert, sondern auch aus Werbungen welche auf den Fahrrädern angezeigt werden. Inwieweit oder ob das System durch die Stadt Wien subventioniert wird ist nicht bekannt. 

Wir untersuchen nun die Einnahmen welche durch die Nutzergemeinde des Systems entsteht. Dies ist durch die Kenntniss des Kostenmodells und der Länge der einzelnen Fahrten möglich.